<a href="https://colab.research.google.com/github/yonasyifter/Account-Storage/blob/master/DL_NN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [14]:

# Download latest version
path = kagglehub.dataset_download("zeeshier/weather-forecast-dataset")

print("Path to dataset files:", path)

100%|██████████| 114k/114k [00:00<00:00, 40.5MB/s]

Extracting files...
Path to dataset files: /root/.cache/kagglehub/datasets/zeeshier/weather-forecast-dataset/versions/1


In [15]:
import pandas as pd

In [16]:
import os

# List all files in the downloaded directory
files_in_dataset = os.listdir(path)

# Filter for CSV files
csv_files = [f for f in files_in_dataset if f.endswith('.csv')]

if not csv_files:
    raise FileNotFoundError(f"No CSV files found in the dataset directory: {path}")

# Assuming the first CSV file found is the correct one for simplicity.
# If there are multiple CSVs and specific one is needed, further logic may be required.
first_csv_file = csv_files[0]
full_csv_path = os.path.join(path, first_csv_file)

df = pd.read_csv(full_csv_path)

print(f"Successfully loaded '{first_csv_file}'.")

Successfully loaded 'weather_forecast_data.csv'.


In [17]:
df.head()

,Temperature,Humidity,Wind_Speed,Cloud_Cover,Pressure,Rain
0,23.720338,89.592641,7.335604,50.501694,1032.378759,rain
1,27.879734,46.489704,5.952484,4.990053,992.614190,no rain
2,25.069084,83.072843,1.371992,14.855784,1007.231620,no rain
3,23.622080,74.367758,7.050551,67.255282,982.632013,rain
4,20.591370,96.858822,4.643921,47.676444,980.825142,no rain


In [18]:
df.isnull().sum()

,0
Temperature,0
Humidity,0
Wind_Speed,0
Cloud_Cover,0
Pressure,0
Rain,0


In [20]:
df = df.dropna()

In [22]:
import tensorflow as tf
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split


In [29]:
X = df[[
    "Temperature",
    "Humidity",
    "Wind_Speed",
    "Cloud_Cover",
    "Pressure"
]]

In [31]:
Y = df["Temperature"]

In [32]:
# Normalize
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

In [40]:
X_scaled

array([[ 0.1554307 ,  1.26539348, -0.44481385,  0.02897213,  0.8947143 ],
       [ 0.72322527, -0.89507374, -0.6841429 , -1.53407371, -1.07457032],
       [ 0.33954662,  0.93859882, -1.47673073, -1.19524633, -0.35066253],
       ...,
       [ 0.75442305, -1.0290299 , -1.21282806,  0.89928899, -0.75241945],
       [-1.0637386 , -0.32274425, -1.3032236 , -1.6237562 ,  1.59414185],
       [ 0.54229936,  1.64176416,  1.49793905,  1.08868017, -0.62316338]])

In [35]:
# Train-test split
X_train, X_test, y_train, y_test = train_test_split(
    X_scaled, Y, test_size=0.2, shuffle=False
)

# Small NN (TFLite optimized)
model = tf.keras.Sequential([
    tf.keras.layers.Dense(16, activation="relu", input_shape=(X_train.shape[1],)),
    tf.keras.layers.Dense(8, activation="relu"),
    tf.keras.layers.Dense(1)
])

/usr/local/lib/python3.12/dist-packages/keras/src/layers/core/dense.py:93: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


In [37]:
# Small NN (TFLite optimized)
model = tf.keras.Sequential([
    tf.keras.layers.Dense(16, activation="relu", input_shape=(X_train.shape[1],)),
    tf.keras.layers.Dense(8, activation="relu"),
    tf.keras.layers.Dense(1)
])

model.compile(
    optimizer="adam",
    loss="mse",
    metrics=["mae"]
)

model.fit(
    X_train, y_train,
    epochs=30,
    batch_size=8,
    validation_data=(X_test, y_test),
    verbose=1
)

Epoch 1/30
250/250 ━━━━━━━━━━━━━━━━━━━━ 3s 4ms/step - loss: 510.3724 - mae: 21.2766 - val_loss: 169.2616 - val_mae: 11.4075
Epoch 2/30
250/250 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - loss: 94.5327 - mae: 7.7215 - val_loss: 10.0011 - val_mae: 2.5211
Epoch 3/30
250/250 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - loss: 9.4069 - mae: 2.4580 - val_loss: 7.2669 - val_mae: 2.1258
Epoch 4/30
250/250 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - loss: 6.9565 - mae: 2.0959 - val_loss: 5.9887 - val_mae: 1.9100
Epoch 5/30
250/250 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - loss: 6.4126 - mae: 2.0114 - val_loss: 4.9540 - val_mae: 1.7352
Epoch 6/30
250/250 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - loss: 5.0244 - mae: 1.7709 - val_loss: 4.1135 - val_mae: 1.5780
Epoch 7/30
250/250 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - loss: 4.1098 - mae: 1.6204 - val_loss: 3.4175 - val_mae: 1.4341
Epoch 8/30
250/250 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - loss: 3.6840 - mae: 1.5228 - val_loss: 2.8176 - val_mae: 1.3002
Epoch 9/30
250/250 ━━━━━━━━━━━━━━━━━━━━ 1s 4ms/s